<a href="https://colab.research.google.com/github/shmouses/EELSpecNet/blob/main/general_GPU_10D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import os 
import tensorflow_datasets as tfds
import copy
import numpy as np
from scipy import signal
from scipy.special import wofz, erf
import matplotlib.pyplot as plt
import notebook
from tqdm.auto import tqdm
import random
import os
import pandas as pd

import notebook
from bokeh.plotting import figure, show
from bokeh.io import output_notebook 
output_notebook()


import tensorflow.experimental.numpy as tnp

In [ ]:
%tensorflow_version 2.x

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
class EELSpecNetModel_CNN_10D(tf.keras.Model):
    def __init__(self, ene_dim):
        super(EELSpecNetModel_CNN_10D, self).__init__()
        
        kerl_size = 4
 
        self.conv_1024x64 = tf.keras.layers.Conv2D(64, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu', padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_512x128 = tf.keras.layers.Conv2D(128, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu', padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_256x256 = tf.keras.layers.Conv2D(256, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu',padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_128x512 = tf.keras.layers.Conv2D(512, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu', padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_64x1024 = tf.keras.layers.Conv2D(1024, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu', padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_32x2048 = tf.keras.layers.Conv2D(2048, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu',padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_16x2048 = tf.keras.layers.Conv2D(2048, (1,kerl_size), strides = (1,2),
                                                   activation = 'relu', padding = 'same',
                                                   kernel_initializer='random_uniform')
 
        self.conv_8x2048 = tf.keras.layers.Conv2D(2048, (1,kerl_size), strides = (1,2),
                                                  activation = 'relu', padding = 'same',
                                                  kernel_initializer='random_uniform')
 
        self.conv_4x2048 = tf.keras.layers.Conv2D(2048, (1,kerl_size), strides = (1,2),
                                                  activation = 'relu',padding = 'same',
                                                  kernel_initializer='random_uniform')
 
        self.conv_2x2048 = tf.keras.layers.Conv2D(2048, (1,kerl_size), strides = (1,2),
                                                  activation = 'relu', padding = 'same',
                                                  kernel_initializer='random_uniform')
        
        #=======================================================================
 
        self.deconv_4x2048 = tf.keras.layers.Conv2DTranspose(2048, (1,kerl_size), strides = (1,2),
                                                             activation = 'relu', padding = 'same',
                                                             kernel_initializer='random_uniform')
 
        self.deconv_8x2048 = tf.keras.layers.Conv2DTranspose(2048, (1,kerl_size), strides = (1,2),
                                                             activation = 'relu', padding = 'same',
                                                             kernel_initializer='random_uniform')
 
        self.deconv_16x2048 = tf.keras.layers.Conv2DTranspose(2048, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
 
        self.deconv_32x2048 = tf.keras.layers.Conv2DTranspose(2048, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
 
        self.deconv_64x1024 = tf.keras.layers.Conv2DTranspose(1024, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
 
        self.deconv_128x512 = tf.keras.layers.Conv2DTranspose(512, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
 
        self.deconv_256x256 = tf.keras.layers.Conv2DTranspose(256, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
        
        self.deconv_512x128 = tf.keras.layers.Conv2DTranspose(128, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
        
        self.deconv_1024x64 = tf.keras.layers.Conv2DTranspose(64, (1,kerl_size), strides = (1,2),
                                                              activation = 'relu', padding = 'same',
                                                              kernel_initializer='random_uniform')
        
        self.deconv_2048x1 = tf.keras.layers.Conv2DTranspose(1, (1,kerl_size), strides = (1,2),
                                                             activation = 'tanh', padding = 'same',
                                                             kernel_initializer='random_uniform')
        
        self.concat = tf.keras.layers.concatenate
        self.relu = tf.keras.activations.relu
        
    
    
    def call(self, inputs):
        
        enc_1024x64 = self.conv_1024x64(inputs)
        
        enc_512x128 = self.conv_512x128(enc_1024x64)
        
        enc_256x256 = self.conv_256x256(enc_512x128)
        
        enc_128x512 = self.conv_128x512(enc_256x256)
        
        enc_64x1024 = self.conv_64x1024(enc_128x512)
        
        enc_32x2048 = self.conv_32x2048(enc_64x1024)
        
        enc_16x2048 = self.conv_16x2048(enc_32x2048)
        
        enc_8x2048 = self.conv_8x2048(enc_16x2048)
        
        enc_4x2048 = self.conv_4x2048(enc_8x2048)
                
        enc_2x2048 = self.conv_2x2048(enc_4x2048)
        
        #=======================================================================
 
        dcd_4x2048 = self.deconv_4x2048(enc_2x2048)  
        dcd_4x2048x2 = self.concat([dcd_4x2048, enc_4x2048], axis=-1)
        
        dcd_8x2048 = self.deconv_8x2048(dcd_4x2048x2)
        dcd_8x2048x2 = self.concat([dcd_8x2048, enc_8x2048], axis=-1)
                
        dcd_16x2048 = self.deconv_16x2048(dcd_8x2048x2)
        dcd_16x2048x2 = self.concat([dcd_16x2048, enc_16x2048], axis=-1)
        
        dcd_32x2048 = self.deconv_32x2048(dcd_16x2048x2)
        dcd_32x2048x2 = self.concat([dcd_32x2048, enc_32x2048], axis=-1)
        
        dcd_64x1024 = self.deconv_64x1024(dcd_32x2048x2)
        dcd_64x1024x2 = self.concat([dcd_64x1024, enc_64x1024], axis=-1)
        
        dcd_128x512 = self.deconv_128x512(dcd_64x1024x2)
        dcd_128x512x2 = self.concat([dcd_128x512, enc_128x512], axis=-1)
        
        dcd_256x256 = self.deconv_256x256(dcd_128x512x2)
        dcd_256x256x2 = self.concat([dcd_256x256, enc_256x256], axis=-1)
        
        dcd_512x128 = self.deconv_512x128(dcd_256x256x2)
        dcd_512x128x2 = self.concat([dcd_512x128, enc_512x128], axis=-1)
        
        dcd_1024x64 = self.deconv_1024x64(dcd_512x128x2)
        dcd_1024x64x2 = self.concat([dcd_1024x64, enc_1024x64], axis=-1)
        
        dcd_2048x1 = self.deconv_2048x1(dcd_1024x64x2)
 
        
        return(dcd_2048x1)

In [ ]:
#constant_optimizer_10d = tf.keras.optimizers.Adam(learning_rate = 5e-5) 

model1 = EELSpecNetModel_CNN_10D(2048)
model1.compile(optimizer = "Adam", loss = 'BinaryCrossentropy', metrics = ['mape','mse'])
model1.build((5000,1,2048,1))
model1.summary()

Model: "eel_spec_net_model_cnn_10d"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  32896     
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  131328    
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  524800    
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  2098176   
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  8390656   
_________________________________________________________________
conv2d_6 (Conv2D)            multiple   

In [ ]:
import tensorflow.experimental.numpy as tnp
tnp.experimental_enable_numpy_behavior()

si_size = [1,2048]

data_original_loaded = np.load("/content/drive/MyDrive/EELSpecNet/Scientific Report Version/GPU/general_training/original_general_6000.npy")
print(np.shape(data_original_loaded))

tnp_original_loaded = tnp.asarray(data_original_loaded)
print(tnp_original_loaded.dtype)
print(tnp_original_loaded.shape)

data_convolved_loaded = np.load("/content/drive/MyDrive/EELSpecNet/Scientific Report Version/GPU/general_training/convolved_general_6000.npy")
print(np.shape(data_convolved_loaded))

tnp_convolved_loaded = tnp.asarray(data_convolved_loaded)
print(tnp_convolved_loaded.dtype)
print(tnp_convolved_loaded.shape)


(6000, 2048)
<dtype: 'float64'>
(6000, 2048)
(6000, 2048)
<dtype: 'float64'>
(6000, 2048)


In [ ]:
x_dim, e_dim = np.shape(data_original_loaded)
print (e_dim)


tnp_original_loaded += 0.001      
tnp_convolved_loaded += 0.001   

tnp_data_original = tnp_original_loaded.reshape((x_dim, 1, e_dim, 1))
tnp_data_convolved = tnp_convolved_loaded.reshape((x_dim, 1, e_dim, 1))

#trainset_size = 5000

tnp_train_original = tnp_data_original[:, :, :, :]
tnp_train_convolved = tnp_data_convolved[:, :, :, :] 

#tnp_test_original = tnp_data_original[trainset_size:, :, :, :]
#tnp_test_convolved = tnp_data_convolved[trainset_size:, :, :, :]

print(np.shape(tnp_train_original))


#print(np.shape(tnp_test_original))

2048
(6000, 1, 2048, 1)


In [ ]:
#constant_optimizer_10d = tf.keras.optimizers.Adam(learning_rate = 5e-5) 


model = EELSpecNetModel_CNN_10D(2048)
op = tf.keras.optimizers.Adam(learning_rate = 5e-5)
model.compile(optimizer = op, loss = 'BinaryCrossentropy', metrics = ['mape','mse'])
  

history= model.fit(tnp_train_convolved, tnp_train_original, validation_split=0.16, batch_size= 16, epochs = 1000)


#Saving model:
model.save('/content/drive/MyDrive/EELSpecNet/Scientific Report Version/GPU/general_saved_model/general_6000')

#Saving training history to csv:
hist_df = pd.DataFrame(history.history) 
hist_csv_file = '/content/drive/MyDrive/EELSpecNet/Scientific Report Version/GPU/general_saved_model/history_general_6000.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

#Saving training history to numpy:
np.save('/content/drive/MyDrive/EELSpecNet/Scientific Report Version/GPU/general_saved_model/Numpy_history_general_6000.npy', history.history) 

print("------------------------Done------------------------")


Epoch 1/1000
315/315 [==============================] - 68s 158ms/step - loss: 0.0100 - mape: 18.1082 - mse: 1.2314e-05 - val_loss: 0.0097 - val_mape: 8.0915 - val_mse: 2.0340e-06
Epoch 2/1000
315/315 [==============================] - 50s 157ms/step - loss: 0.0097 - mape: 8.0734 - mse: 2.1077e-06 - val_loss: 0.0097 - val_mape: 7.5504 - val_mse: 2.0313e-06
Epoch 3/1000
315/315 [==============================] - 49s 157ms/step - loss: 0.0097 - mape: 8.1527 - mse: 2.1165e-06 - val_loss: 0.0097 - val_mape: 7.7439 - val_mse: 2.0066e-06
Epoch 4/1000
315/315 [==============================] - 49s 157ms/step - loss: 0.0098 - mape: 12.4428 - mse: 4.4436e-06 - val_loss: 0.0098 - val_mape: 10.1875 - val_mse: 2.7224e-06
Epoch 5/1000
315/315 [==============================] - 49s 157ms/step - loss: 0.0098 - mape: 8.6019 - mse: 2.2225e-06 - val_loss: 0.0097 - val_mape: 8.4653 - val_mse: 2.1388e-06
Epoch 6/1000
315/315 [==============================] - 49s 157ms/step - loss: 0.0098 - mape: 8.4983 -

In [ ]:
#print(tnp_test_convolved.shape)

In [ ]:
#y=model.predict(tnp_test_convolved[:,:,:,:].reshape(1000, 1, 2048, 1))
#np.save('/content/drive/MyDrive/EELSpecNet/Scientific Report Version/Output_level1/Numpy_original.npy',tnp_test_original[:,:,:,:])
#np.save('/content/drive/MyDrive/EELSpecNet/Scientific Report Version/Output_level1/Numpy_convolved.npy',tnp_test_convolved[:,:,:,:])
#np.save('/content/drive/MyDrive/EELSpecNet/Scientific Report Version/Output_level1/Numpy_predict.npy', y) 

#plt.plot(y[598,0,300:400,0])
#plt.plot(tnp_test_original[598,0,300:400,0]) 

In [ ]:
#plt.plot(y[598,0,430:600,0])
#plt.plot(tnp_test_original[598,0,430:600,0])

In [ ]:
#fig= figure()

#xx = np.linspace(0,2048,2048)
#o = tnp_test_original.numpy
#cc= tnp_test_convolved.numpy

#fig.multi_line([xx, xx], [oo, cc], color=["firebrick", "blue"], line_width=2)

#show(fig)